In [ ]:
# Instalar Java Tool Kit
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixar Spark
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompactar Spark
!tar xf /content/spark-3.4.1-bin-hadoop3.tgz

# Instalar lib findspark que ajuda a localizar o Spark no sistema e importa-lo como uma biblioteca regular
!pip install -q findspark

In [ ]:
# Lib os - Permite Interagir com S.O - Cria variaveis de ambiente necessarias
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

In [ ]:
# Inicializa Findspark - Facilita Uso de Spark em Notebook
import findspark
findspark.init()

In [ ]:
# Start SparkContext e StreamingContext
from pyspark.context import SparkContext
from pyspark.streaming import StreamingContext
from time import sleep

sc = SparkContext(appName="DStream Example")
ssc = StreamingContext(sc,5)  # Define Leitura dos Dados a Cada 5 Segundos

In [ ]:
# Gera Objeto DStream Definindo como Fonte de Entrada a Pasta Streaming
lines = ssc.textFileStream('Streaming')

In [ ]:
# Executa Transformações flatMap / map / reduceByKey / sortByKey Sobre Dados Lidos
words = lines.flatMap(lambda x:x.split(" "))
mapped_words  = words.map(lambda x:(x,1))
reduced_words = mapped_words.reduceByKey(lambda x,y:x+y)
sorted_words  = reduced_words.map(lambda x:(x[1],x[0])).transform(lambda x:x.sortByKey(False))

In [ ]:
# Executa Ação pprint Sobre Objeto DStream
sorted_words.pprint()

In [ ]:
# Inicia Processamento Streaming
ssc.start()

# Aguarda 20 Segundos antes do Stop
sleep(20)

# stopSparkContext -> True para Contexto Spark alem do Contexto Streaming
# stopGraceFully   -> True aguarda processamento pendente concluir antes de interromper
ssc.stop(stopSparkContext=True, stopGraceFully=True)

-------------------------------------------
Time: 2023-11-03 19:33:25
-------------------------------------------

-------------------------------------------
Time: 2023-11-03 19:33:30
-------------------------------------------
(4, 'Data')
(3, 'Map')
(2, 'Python')
(2, 'Java')
(2, 'Hadoop')
(2, 'Scala')
(1, 'Warehouse')
(1, 'Hive')
(1, 'Spark')
(1, 'Lake')
...

-------------------------------------------
Time: 2023-11-03 19:33:35
-------------------------------------------

-------------------------------------------
Time: 2023-11-03 19:33:40
-------------------------------------------

-------------------------------------------
Time: 2023-11-03 19:33:45
-------------------------------------------

-------------------------------------------
Time: 2023-11-03 19:33:50
-------------------------------------------



In [ ]:
# Parar SparkContext - Somente Executar se Precisar Reprocessar o Streaming
ssc.stop(stopSparkContext=True, stopGraceFully=True)